## FINAL Advanced RAG

![structure.png](../Schemes/Advanced_RAG.png)

В этом этапе мы соберем разные варианты улучшенных RAG

## Variant №1
### Html_split + routing + step_back  + rerank

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Advanced_RAG_1_gpt_3.5"

In [ ]:
# Примеры для Few Shot
examples = [
    {"input": "Нужно ли ИП указывать свой сайт для предприятия?", "output": "Что необходимо предоставить ИП?"},
    {"input": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?", "output": "Что будет при неиспользовании кредитной карты?"},
    {"input": "Дают ли кредит на автомобиль?", "output": "На что дают кредит?"},
    {"input": "Распространяется ли страховое покрытие на смерть в алкогольном опьянении?", "output": "На что не распространяется страховое покрытие по риску 'Смерть'?"}
]

example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}")])
few_shot_prompt = FewShotChatMessagePromptTemplate(example_prompt=example_prompt, examples=examples)

step_back_prompt = ChatPromptTemplate.from_messages([
    ("system", """Вы являетесь экспертом в области мировых знаний. Ваша задача - сделать шаг назад и перефразировать вопрос в более общий, на который проще ответить. Вот несколько примеров:"""),
    few_shot_prompt,
    ("user", "{question}")
])

# Инициализация моделей
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

# краткое содержание документов
about_docs = {
    '1_Безопасное использование банковской карты.pdf': """Этот документ представляет собой руководство по безопасному использованию банковских карт от Тинькофф Банка. В нем содержатся рекомендации по предотвращению мошенничества, защите конфиденциальной информации о карте и PIN-коде, а также советы по совершению различных операций с банковской картой (оплата покупок, снятие наличных, бронирование гостиниц и аренда автомобилей). Документ также описывает распространенные схемы мошенничества, такие как фишинг, и способы, которыми пользуются мошенники для кражи данных карты. Если вы планируете задавать вопросы, касающиеся безопасного использования банковских карт, предотвращения мошенничества или правил совершения операций с картой, этот документ будет релевантным источником информации.""",
    '2_Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального догов.pdf': """Этот документ представляет собой "Перечень документов и сведений, предоставляемых клиентом (представителем клиента) для присоединения к условиям Универсального договора, а также в целях обслуживания в рамках Универсального договора". В нем подробно описывается перечень документов и сведений, которые должны предоставлять различные категории клиентов (физические лица, юридические лица, индивидуальные предприниматели, нотариусы, адвокаты, арбитражные управляющие, иностранные граждане и др.) для идентификации, открытия счетов и обслуживания в банке. Документ содержит требования к формату и срокам действия предоставляемых документов, а также особенности идентификации нерезидентов, бенефициарных владельцев, представителей и выгодоприобретателей. Это подробное руководство будет полезно при взаимодействии с банком для уточнения пакета необходимых документов.""",
    '3_Пользовательское соглашение АО «Тинькофф Страхование».pdf': """Этот документ представляет собой пользовательское соглашение АО "Тинькофф Страхование", определяющее условия использования материалов и сервисов на их официальном сайте и других дистанционных каналах взаимодействия. В нем описываются правила дистанционного взаимодействия со страховщиком, использования простой электронной подписи, обработки персональных данных, а также обязательства пользователей и прочие условия. Документ содержит юридические определения и разъяснения, относящиеся к процессу заключения и сопровождения договоров страхования в электронном виде.""",
    '4_Покупка:продажа слитков драгоценных металлов.pdf': """ Этот документ представляет собой Правила оказания сервиса «Покупка/продажа слитков драгоценных металлов» в АО «Тинькофф Банк». В документе описаны условия и порядок совершения сделок купли-продажи слитков драгметаллов между банком и клиентом, также условия передачи слитков на ответственное хранение в банк и их возврата клиенту, подтверждение получения слитка фотосъемкой клиента. Документ содержит определения основных терминов, процедуры оформления заявок, доставки слитков, подписания актов приема-передачи и рекламационные процедуры. Также приведены права и обязанности сторон, ответственность и заключительные положения, срок выдачи доверенности, назначение доверенных лиц. """,
    '5_Условия комплексного банковского обслуживания физических лиц.pdf': """Документ «Условия комплексного банковского обслуживания физических лиц» от банка Тинькофф содержит подробные условия предоставления различных банковских услуг для физических лиц. В документе определены ключевые термины и описаны виды договоров, которые могут заключаться между клиентами и банком. Основное внимание уделено правилам и порядку использования дистанционных сервисов, таким как интернет-банк и мобильное приложение, а также мерам безопасности при использовании банковских продуктов. Описаны права и обязанности сторон, условия открытия, ведения и закрытия счетов, накопительные счета, вклады (открытие вклада - условия, возраст), кредитные и расчетные карты, порядок расчетов и требования к идентификации клиентов, страховые выплаты в случае несчастных случаев, выдача кредитов, выплаты по кредитам, расторжение договора в случае неиспользования кредитной карты и возврат денег при использовании кредитки. Документ также включает информацию о применении простой электронной подписи и использовании различных электронных устройств для взаимодействия с банком.
        Ключевые слова по разделам и подразделам:
        - Термины и определения: абонентский номер, абонентское устройство, аутентификационные данные, банкомат, виртуальный голосовой помощник, вклад срочный, вклад до востребования, выписка, дистанционное обслуживание, договор.
        - Виды договоров: договор вклада, договор кредитной карты, договор расчетной карты, договор реструктуризации задолженности, кредитный договор, договор залога.
        - Устройства и сервисы: интернет-банк, мобильное приложение, мобильный банк, картсчет.
        - Процедуры и условия: операционное время, перечисление денежных средств, ПИН-код, платежное приложение, простая электронная подпись, предмет залога, расчетная карта, расчетный период.
        - Клиенты и идентификация: клиент, код доступа, компрометация, права требования, сообщение, счет вклада, тарифный план.
        - Дополнительная информация:  открытие вкладов, возврат средств по кредитной карте """
}


summary_docs = [Document(page_content=val, metadata={'keys': key}) for key, val in about_docs.items()]
vectorstore_summary = Chroma.from_documents(documents=summary_docs, embedding=embeddings_model, collection_name="summaries")
retriever_summary = vectorstore_summary.as_retriever(search_kwargs={'k': 1})

splits = split_documents_html(chunk_size=1800, chunk_overlap=100, flag=False)

vectorstore_1 = Chroma.from_documents(documents=splits[0], embedding=embeddings_model, collection_name="vectorstore_1")
vectorstore_2 = Chroma.from_documents(documents=splits[2], embedding=embeddings_model, collection_name="vectorstore_3")
vectorstore_3 = Chroma.from_documents(documents=splits[3], embedding=embeddings_model, collection_name="vectorstore_4")
vectorstore_4 = Chroma.from_documents(documents=splits[1], embedding=embeddings_model, collection_name="vectorstore_2")
vectorstore_5 = Chroma.from_documents(documents=splits[4], embedding=embeddings_model, collection_name="vectorstore_5")

def determine_docs(summary_text):
    k = 25
    if summary_text == '1_Безопасное использование банковской карты.pdf':
        return vectorstore_1.as_retriever(search_kwargs={'k': k})
    elif summary_text == '2_Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального догов.pdf':
        return vectorstore_2.as_retriever(search_kwargs={'k': k})
    elif summary_text == '3_Пользовательское соглашение АО «Тинькофф Страхование».pdf':
        return vectorstore_3.as_retriever(search_kwargs={'k': k})
    elif summary_text == '4_Покупка:продажа слитков драгоценных металлов.pdf':
        return vectorstore_4.as_retriever(search_kwargs={'k': k})
    else:
        return vectorstore_5.as_retriever(search_kwargs={'k': k})

# Создание функции для выполнения полной цепочки поиска
def retrieval_chain(question):
    summary_text = retriever_summary.invoke(question)[0].metadata['keys']
    retriever = determine_docs(summary_text)
    return retriever.invoke(question)

In [ ]:
# Определяем функцию для ранжирования документов
def rerank_retriever(query):
    documents = retrieval_chain(query)
    return rerank_doc(query, top_n=2, documents=documents)

# Генерация перефразированных вопросов
generate_queries_step_back = step_back_prompt | llm | StrOutputParser()

# Итоговый промт
final_prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
    Я собираюсь задать вам вопрос. Пожалуйста, выберите из контекста те части, которые, по вашему мнению, могут помочь ответить на вопрос, и основываясь на них, постройте свой ответ.\n
    Ваш ответ должен быть исчерпывающим, структурированным и не противоречить следующему контексту, если он уместен.\n
    Если контекст не даёт достаточно информации для точного ответа, сообщите клиенту, что вы не знаете, и предложите обратиться в банк для получения дополнительной информации.\n
    Отвечай как оператор подерджки банка Tinkoff, старайся отвечать не слишком длинно, отвечай только по делу и сразу говори ответ без предварительных рассуждений текстовых.

    Question: {question} 
    Context: {context} 
    Step_back_context: {step_back_context}
    Answer:
""")


# Объединение всех шагов в одну цепочку
final_chain_1 = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | rerank_retriever,
        "step_back_context": generate_queries_step_back | rerank_retriever,
        "question": RunnablePassthrough()
    }
    | final_prompt
    | llm
    | StrOutputParser()
)

# # Пример вызова
# response = final_chain_1.invoke({"question": "Можно ли открыть вклад в 12 лет?"})
# print(response)

#### При рассмотрении внимательно обратить внимание, кто лучший контекст предоставляет 

### Загрузка Датасета

In [ ]:
import pandas as pd
import time
df = pd.read_csv('../dataset.csv')
# Функция для обработки одного запроса с учетом задержки и повторных попыток
def process_question(question, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            answer = final_chain_1.invoke({"question": question})
            time.sleep(6)  # Ждать n секунд перед следующим запросом
            return answer
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            attempts += 1
            time.sleep(5)  # Подождать перед повторной попыткой
    print("Failed after multiple attempts.")
    return None

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))
display(df)


In [ ]:
df.to_csv('../rag_answer/advanced_rag/html_split/Advanced_RAG_1_gpt_3.5.csv', index=False)

### GIGA-Chat - 6 блокировок, среднее время около 6.26 s( от 4.74 - 11.06) контекст видит плохо, галлюцинирует 

## Variant №2
### Html_split + routing + step_back  + compressor_rerank

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Advanced_RAG_2_gpt_3.5"

In [ ]:
# Примеры для Few Shot
examples = [
    {"input": "Нужно ли ИП указывать свой сайт для предприятия?", "output": "Что необходимо предоставить ИП?"},
    {"input": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?", "output": "Что будет при неиспользовании кредитной карты?"},
    {"input": "Дают ли кредит на автомобиль?", "output": "На что дают кредит?"},
    {"input": "Распространяется ли страховое покрытие на смерть в алкогольном опьянении?", "output": "На что не распространяется страховое покрытие по риску 'Смерть'?"}
]

example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}")])
few_shot_prompt = FewShotChatMessagePromptTemplate(example_prompt=example_prompt, examples=examples)

step_back_prompt = ChatPromptTemplate.from_messages([
    ("system", """Вы являетесь экспертом в области мировых знаний. Ваша задача - сделать шаг назад и перефразировать вопрос в более общий, на который проще ответить. Вот несколько примеров:"""),
    few_shot_prompt,
    ("user", "{question}")
])

# Инициализация моделей
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

# краткое содержание документов
about_docs = {
    '5_Условия комплексного банковского обслуживания физических лиц.pdf': """Документ «Условия комплексного банковского обслуживания физических лиц» от банка Тинькофф содержит подробные условия предоставления различных банковских услуг для физических лиц. В документе определены ключевые термины и описаны виды договоров, которые могут заключаться между клиентами и банком. Основное внимание уделено правилам и порядку использования дистанционных сервисов, таким как интернет-банк и мобильное приложение, а также мерам безопасности при использовании банковских продуктов. Описаны права и обязанности сторон, условия открытия, ведения и закрытия счетов, накопительные счета, вклады (открытие вклада - условия, возраст), кредитные и расчетные карты, порядок расчетов и требования к идентификации клиентов, страховые выплаты в случае несчастных случаев, выдача кредитов, выплаты по кредитам, расторжение договора в случае неиспользования кредитной карты и возврат денег при использовании кредитки. Документ также включает информацию о применении простой электронной подписи и использовании различных электронных устройств для взаимодействия с банком.
        Ключевые слова по разделам и подразделам:
        - Термины и определения: абонентский номер, абонентское устройство, аутентификационные данные, банкомат, виртуальный голосовой помощник, вклад срочный, вклад до востребования, выписка, дистанционное обслуживание, договор.
        - Виды договоров: договор вклада, договор кредитной карты, договор расчетной карты, договор реструктуризации задолженности, кредитный договор, договор залога.
        - Устройства и сервисы: интернет-банк, мобильное приложение, мобильный банк, картсчет.
        - Процедуры и условия: операционное время, перечисление денежных средств, ПИН-код, платежное приложение, простая электронная подпись, предмет залога, расчетная карта, расчетный период.
        - Клиенты и идентификация: клиент, код доступа, компрометация, права требования, сообщение, счет вклада, тарифный план.
        - Дополнительная информация:  открытие вкладов, возврат средств по кредитной карте """,
    '1_Безопасное использование банковской карты.pdf': """Этот документ представляет собой руководство по безопасному использованию банковских карт от Тинькофф Банка. В нем содержатся рекомендации по предотвращению мошенничества, защите конфиденциальной информации о карте и PIN-коде, а также советы по совершению различных операций с банковской картой (оплата покупок, снятие наличных, бронирование гостиниц и аренда автомобилей). Документ также описывает распространенные схемы мошенничества, такие как фишинг, и способы, которыми пользуются мошенники для кражи данных карты. Если вы планируете задавать вопросы, касающиеся безопасного использования банковских карт, предотвращения мошенничества или правил совершения операций с картой, этот документ будет релевантным источником информации.""",
    '2_Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального догов.pdf': """Этот документ представляет собой "Перечень документов и сведений, предоставляемых клиентом (представителем клиента) для присоединения к условиям Универсального договора, а также в целях обслуживания в рамках Универсального договора". В нем подробно описывается перечень документов и сведений, которые должны предоставлять различные категории клиентов (физические лица, юридические лица, индивидуальные предприниматели, нотариусы, адвокаты, арбитражные управляющие, иностранные граждане и др.) для идентификации, открытия счетов и обслуживания в банке. Документ содержит требования к формату и срокам действия предоставляемых документов, а также особенности идентификации нерезидентов, бенефициарных владельцев, представителей и выгодоприобретателей. Это подробное руководство будет полезно при взаимодействии с банком для уточнения пакета необходимых документов.""",
    '3_Пользовательское соглашение АО «Тинькофф Страхование».pdf': """Этот документ представляет собой пользовательское соглашение АО "Тинькофф Страхование", определяющее условия использования материалов и сервисов на их официальном сайте и других дистанционных каналах взаимодействия. В нем описываются правила дистанционного взаимодействия со страховщиком, использования простой электронной подписи, обработки персональных данных, а также обязательства пользователей и прочие условия. Документ содержит юридические определения и разъяснения, относящиеся к процессу заключения и сопровождения договоров страхования в электронном виде.""",
    '4_Покупка:продажа слитков драгоценных металлов.pdf': """ Этот документ представляет собой Правила оказания сервиса «Покупка/продажа слитков драгоценных металлов» в АО «Тинькофф Банк». В документе описаны условия и порядок совершения сделок купли-продажи слитков драгметаллов между банком и клиентом, также условия передачи слитков на ответственное хранение в банк и их возврата клиенту, подтверждение получения слитка фотосъемкой клиента. Документ содержит определения основных терминов, процедуры оформления заявок, доставки слитков, подписания актов приема-передачи и рекламационные процедуры. Также приведены права и обязанности сторон, ответственность и заключительные положения, срок выдачи доверенности, назначение доверенных лиц. """
    
}

summary_docs = [Document(page_content=val, metadata={'keys': key}) for key, val in about_docs.items()]
vectorstore_summary = Chroma.from_documents(documents=summary_docs, embedding=embeddings_model, collection_name="summaries")
retriever_summary = vectorstore_summary.as_retriever(search_kwargs={'k': 1})

splits = split_documents_html(chunk_size=1800, chunk_overlap=100, flag=False)

vectorstore_1 = Chroma.from_documents(documents=splits[0], embedding=embeddings_model, collection_name="vectorstore_1")
vectorstore_2 = Chroma.from_documents(documents=splits[2], embedding=embeddings_model, collection_name="vectorstore_3")
vectorstore_3 = Chroma.from_documents(documents=splits[3], embedding=embeddings_model, collection_name="vectorstore_4")
vectorstore_4 = Chroma.from_documents(documents=splits[1], embedding=embeddings_model, collection_name="vectorstore_2")
vectorstore_5 = Chroma.from_documents(documents=splits[4], embedding=embeddings_model, collection_name="vectorstore_5")

def determine_docs(summary_text):
    k = 25
    if summary_text == '1_Безопасное использование банковской карты.pdf':
        return vectorstore_1.as_retriever(search_kwargs={'k': k})
    elif summary_text == '2_Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального догов.pdf':
        return vectorstore_2.as_retriever(search_kwargs={'k': k})
    elif summary_text == '3_Пользовательское соглашение АО «Тинькофф Страхование».pdf':
        return vectorstore_3.as_retriever(search_kwargs={'k': k})
    elif summary_text == '4_Покупка:продажа слитков драгоценных металлов.pdf':
        return vectorstore_4.as_retriever(search_kwargs={'k': k})
    else:
        return vectorstore_5.as_retriever(search_kwargs={'k': k})

In [ ]:
from langchain.retrievers import  ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

# Создание функции для выполнения полной цепочки поиска
def retrieval_chain(question):
    summary_text = retriever_summary.invoke(question)[0].metadata['keys']
    retriever = determine_docs(summary_text)
    compressor = CohereRerank()
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever)
    # Выполнение поиска с компрессией и возвращение только 2 результатов
    results = get_unique_documents(compression_retriever.invoke(question))
    return results[:2]

# Генерация перефразированных вопросов
generate_queries_step_back = step_back_prompt | llm | StrOutputParser()

# Итоговый промт
final_prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
    Я собираюсь задать вам вопрос. Пожалуйста, выберите из контекста те части, которые, по вашему мнению, могут помочь ответить на вопрос, и основываясь на них, постройте свой ответ.\n
    Ваш ответ должен быть исчерпывающим, структурированным и не противоречить следующему контексту, если он уместен.\n
    Если контекст не даёт достаточно информации для точного ответа, сообщите клиенту, что вы не знаете, и предложите обратиться в банк для получения дополнительной информации.\n
    Отвечай как оператор подерджки банка Tinkoff, старайся отвечать не слишком длинно, отвечай только по делу и сразу говори ответ без предварительных рассуждений текстовых.

    Question: {question} 
    Context: {context} 
    Answer:
""")

def remove_duplicate_contexts(question, context, step_back_context) -> str:
    all_contexts = context + step_back_context
    final_context = get_unique_documents(all_contexts)
    
    prompt = final_prompt.format(question=question, context=final_context)
    return prompt

# Объединение всех шагов в одну цепочку
final_chain_2 = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retrieval_chain,
        "step_back_context": generate_queries_step_back | retrieval_chain,
        "question": RunnablePassthrough()
    }
    | RunnableLambda(lambda x: remove_duplicate_contexts(x["question"], x["context"], x["step_back_context"]))
    | llm
    | StrOutputParser()
)

# # Пример вызова
# response = final_chain_2.invoke({"question": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?"})
# print(response)

### Загрузка Датасета

In [ ]:
import pandas as pd
import time
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки и повторных попыток
def process_question(question, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            answer = final_chain_2.invoke({"question": question})
            time.sleep(6)  # Ждать n секунд перед следующим запросом
            return answer
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            attempts += 1
            time.sleep(15)  # Подождать перед повторной попыткой
    print("Failed after multiple attempts.")
    return None

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))
display(df)

In [ ]:
df.to_csv('../rag_answer/advanced_rag/html_split/Advanced_RAG_2_gpt_3.5.csv', index=False)

Giga generation stopped with reason: blacklist - 9


## Variant №3
### Html_split  + step_back  + rerank

### Библиотеки и окружения 

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Advanced_RAG_3_gpt_3.5"

In [ ]:
# Примеры для Few Shot
examples = [
    {"input": "Нужно ли ИП указывать свой сайт для предприятия?", "output": "Что необходимо предоставить ИП?"},
    {"input": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?", "output": "Что будет при неиспользовании кредитной карты?"},
    {"input": "Дают ли кредит на автомобиль?", "output": "На что дают кредит?"},
    {"input": "Распространяется ли страховое покрытие на смерть в алкогольном опьянении?", "output": "На что не распространяется страховое покрытие по риску 'Смерть'?"}
]

example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}")])
few_shot_prompt = FewShotChatMessagePromptTemplate(example_prompt=example_prompt, examples=examples)

step_back_prompt = ChatPromptTemplate.from_messages([
    ("system", """Вы являетесь экспертом в области мировых знаний. Ваша задача - сделать шаг назад и перефразировать вопрос в более общий, на который проще ответить. Вот несколько примеров:"""),
    few_shot_prompt,
    ("user", "{question}")
])

# Инициализация моделей
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

splits = split_documents_html(chunk_size=1800, chunk_overlap=100)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)
# Определяем функцию для ранжирования документов
def retrieval_chain(question):
    documents = retriever.invoke(question)
    return rerank_doc(question, top_n=2, documents=documents)


# Генерация перефразированных вопросов
generate_queries_step_back = step_back_prompt | llm | StrOutputParser()

# Итоговый промт
final_prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
    Я собираюсь задать вам вопрос. Пожалуйста, выберите из контекста те части, которые, по вашему мнению, могут помочь ответить на вопрос, и основываясь на них, постройте свой ответ.\n
    Ваш ответ должен быть исчерпывающим, структурированным и не противоречить следующему контексту, если он уместен.\n
    Если контекст не даёт достаточно информации для точного ответа, сообщите клиенту, что вы не знаете, и предложите обратиться в банк для получения дополнительной информации.\n
    Отвечай как оператор подерджки банка Tinkoff, старайся отвечать не слишком длинно, отвечай только по делу и сразу говори ответ без предварительных рассуждений текстовых.

    Question: {question} 
    Context: {context} 
    Answer:
""")

def remove_duplicate_contexts(question, context, step_back_context) -> str:
    all_contexts = context + step_back_context
    final_context = get_unique_documents(all_contexts)
    
    prompt = final_prompt.format(question=question, context=final_context)
    return prompt

# Объединение всех шагов в одну цепочку
final_chain_3 = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retrieval_chain,
        "step_back_context": generate_queries_step_back | retrieval_chain,
        "question": RunnablePassthrough()
    }
    | RunnableLambda(lambda x: remove_duplicate_contexts(x["question"], x["context"], x["step_back_context"]))
    | llm
    | StrOutputParser()
)

# # Пример вызова
# response = final_chain_3.invoke({"question": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?"})
# print(response)

### Загрузка Датасета

In [ ]:
import pandas as pd
import time
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки и повторных попыток
def process_question(question, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            answer = final_chain_3.invoke({"question": question})
            time.sleep(6)  # Ждать n секунд перед следующим запросом
            return answer
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            attempts += 1
            time.sleep(15)  # Подождать перед повторной попыткой
    print("Failed after multiple attempts.")
    return None

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))
display(df)

In [ ]:
df.to_csv('../rag_answer/advanced_rag/html_split/Advanced_RAG_3_gpt_3.5.csv', index=False)

## Variant №4
### Html_split + routing  + rerank 

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "FINAL_4_gpt_3.5_var_2"

In [ ]:
# краткое содержание документов
about_docs = {
    '1_Безопасное использование банковской карты.pdf': """Этот документ представляет собой руководство по безопасному использованию банковских карт от Тинькофф Банка. В нем содержатся рекомендации по предотвращению мошенничества, защите конфиденциальной информации о карте и PIN-коде, а также советы по совершению различных операций с картой (оплата покупок, снятие наличных, бронирование гостиниц и аренда автомобилей). Документ также описывает распространенные схемы мошенничества, такие как фишинг, и способы, которыми пользуются мошенники для кражи данных карты. Если вы планируете задавать вопросы, касающиеся безопасного использования банковских карт, предотвращения мошенничества или правил совершения операций с картой, этот документ будет релевантным источником информации.""",
    '2_Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального догов.pdf': """Этот документ представляет собой "Перечень документов и сведений, предоставляемых клиентом (представителем клиента) для присоединения к условиям Универсального договора, а также в целях обслуживания в рамках Универсального договора". В нем подробно описывается перечень документов и сведений, которые должны предоставлять различные категории клиентов (физические лица, юридические лица, индивидуальные предприниматели, нотариусы, адвокаты, арбитражные управляющие, иностранные граждане и др.) для идентификации, открытия счетов и обслуживания в банке. Документ содержит требования к формату и срокам действия предоставляемых документов, а также особенности идентификации нерезидентов, бенефициарных владельцев, представителей и выгодоприобретателей. Это подробное руководство будет полезно при взаимодействии с банком для уточнения пакета необходимых документов.""",
    '3_Пользовательское соглашение АО «Тинькофф Страхование».pdf': """Этот документ представляет собой пользовательское соглашение АО "Тинькофф Страхование", определяющее условия использования материалов и сервисов на их официальном сайте и других дистанционных каналах взаимодействия. В нем описываются правила дистанционного взаимодействия со страховщиком, использования простой электронной подписи, обработки персональных данных, а также обязательства пользователей и прочие условия. Документ содержит юридические определения и разъяснения, относящиеся к процессу заключения и сопровождения договоров страхования в электронном виде.""",
    '4_Покупка:продажа слитков драгоценных металлов.pdf': """ Этот документ представляет собой Правила оказания сервиса «Покупка/продажа слитков драгоценных металлов» в АО «Тинькофф Банк». В документе описаны условия и порядок совершения сделок купли-продажи слитков драгметаллов между банком и клиентом, также условия передачи слитков на ответственное хранение в банк и их возврата клиенту, подтверждение получения слитка фотосъемкой клиента. Документ содержит определения основных терминов, процедуры оформления заявок, доставки слитков, подписания актов приема-передачи и рекламационные процедуры. Также приведены права и обязанности сторон, ответственность и заключительные положения, срок выдачи доверенности, назначение доверенных лиц. """,
    '5_Условия комплексного банковского обслуживания физических лиц.pdf': """Документ «Условия комплексного банковского обслуживания физических лиц» от банка Тинькофф содержит подробные условия предоставления различных банковских услуг для физических лиц. В документе определены ключевые термины и описаны виды договоров, которые могут заключаться между клиентами и банком. Основное внимание уделено правилам и порядку использования дистанционных сервисов, таким как интернет-банк и мобильное приложение, а также мерам безопасности при использовании банковских продуктов. Описаны права и обязанности сторон, условия открытия, ведения и закрытия счетов, накопительные счета, вклады (открытие вклада - условия, возраст), кредитные и расчетные карты, порядок расчетов и требования к идентификации клиентов, страховые выплаты в случае несчастных случаев, выдача кредитов, выплаты по кредитам, расторжение договора в случае неиспользования кредитной карты и возврат денег при использовании кредитки. Документ также включает информацию о применении простой электронной подписи и использовании различных электронных устройств для взаимодействия с банком.

                                                                            Ключевые слова по разделам и подразделам:

                                                                            Термины и определения: абонентский номер, абонентское устройство, аутентификационные данные, банкомат, виртуальный голосовой помощник, вклад срочный, вклад до востребования, выписка, дистанционное обслуживание, договор.
                                                                            Виды договоров: договор вклада, договор кредитной карты, договор расчетной карты, договор реструктуризации задолженности, кредитный договор, договор залога.
                                                                            Устройства и сервисы: интернет-банк, мобильное приложение, мобильный банк, картсчет.
                                                                            Процедуры и условия: операционное время, перечисление денежных средств, ПИН-код, платежное приложение, простая электронная подпись, предмет залога, расчетная карта, расчетный период.
                                                                            Клиенты и идентификация: клиент, код доступа, компрометация, права требования, сообщение, счет вклада, тарифный план."""
}

summary_docs = [
    Document(page_content=val, metadata={'keys': key})
    for key, val in about_docs.items()
]

# Инициализация моделей
# embeddings_model = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
# llm = GigaChat(model='GigaChat-Plus', verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

# Индексация
vectorstore_summary = Chroma.from_documents(documents=summary_docs, embedding=embeddings_model, collection_name="summaries")

# Устанавливаем ретриевер для поиска контекста
retriever_summary = vectorstore_summary.as_retriever(search_kwargs={'k': 1})

In [ ]:
splits = split_documents_html(chunk_size=1800, chunk_overlap=100, flag=False)

vectorstore_1 = Chroma.from_documents(documents=splits[0], embedding=embeddings_model, collection_name="vectorstore_1")
vectorstore_2 = Chroma.from_documents(documents=splits[2], embedding=embeddings_model, collection_name="vectorstore_3")
vectorstore_3 = Chroma.from_documents(documents=splits[3], embedding=embeddings_model, collection_name="vectorstore_4")
vectorstore_4 = Chroma.from_documents(documents=splits[1], embedding=embeddings_model, collection_name="vectorstore_2")
vectorstore_5 = Chroma.from_documents(documents=splits[4], embedding=embeddings_model, collection_name="vectorstore_5")

In [ ]:
# определить vectorestore для крактого документа (Сопоставить полный документ с кратким содержанием)
def determine_docs(summary_text):
    k = 25
    if summary_text == '1_Безопасное использование банковской карты.pdf':
        return vectorstore_1.as_retriever(search_kwargs={'k': k})
    elif summary_text == '2_Перечень документов и сведений, предоставляемых Клиентом (представителем Клиента) для присоединения к условиям Универсального догов.pdf':
        return vectorstore_2.as_retriever(search_kwargs={'k': k})
    elif summary_text == '3_Пользовательское соглашение АО «Тинькофф Страхование».pdf':
        return vectorstore_3.as_retriever(search_kwargs={'k': k})
    elif summary_text == '4_Покупка:продажа слитков драгоценных металлов.pdf':
        return vectorstore_4.as_retriever(search_kwargs={'k': k})
    else:
        return vectorstore_5.as_retriever(search_kwargs={'k': k})    
    
# Создание функции для выполнения полной цепочки поиска
def retrieval_chain(question):
    # Найти краткое содержание
    summary_text = retriever_summary.invoke(question)[0].metadata['keys']
    
    # Определить соответствующий retriever и найти полный документ
    retriever = determine_docs(summary_text)
    return retriever.invoke(question)

# Создание функции для выполнения полной цепочки поиска
def retrieval_chain(question):
    summary_text = retriever_summary.invoke(question)[0].metadata['keys']
    retriever = determine_docs(summary_text)
    return retriever.invoke(question)

In [ ]:
# Определяем функцию для ранжирования документов
def rerank_retriever(query):
    documents = retrieval_chain(query)
    return rerank_doc(query, top_n=3, documents=documents)

# Итоговый промт
final_prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
    Я собираюсь задать вам вопрос. Пожалуйста, выберите из контекста те части, которые, по вашему мнению, могут помочь ответить на вопрос, и основываясь на них, постройте свой ответ.\n
    Ваш ответ должен быть исчерпывающим, структурированным и не противоречить следующему контексту, если он уместен.\n
    Если контекст не даёт достаточно информации для точного ответа, сообщите клиенту, что вы не знаете, и предложите обратиться в банк для получения дополнительной информации.\n
    Отвечай как оператор подерджки банка Tinkoff, старайся отвечать не слишком длинно, отвечай только по делу и сразу говори ответ без предварительных рассуждений текстовых.

    Question: {question} 
    Context: {context} 
    Answer:

""")

# Объединение всех шагов в одну цепочку
final_rag_chain_4 = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | rerank_retriever,
        "question": RunnablePassthrough()
    }
    | final_prompt
    | llm
    | StrOutputParser()
)

# # Пример вызова
# response = final_rag_chain_4.invoke({"question": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?"})
# print(response)

### Загрузка Датасета

In [ ]:
import pandas as pd
import time

# Загрузка данных
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки и обработкой исключений
def process_question(question):
    max_retries = 3
    retries = 0
    while retries < max_retries:
        try:
            answer = final_rag_chain_4.invoke({"question": question})
            time.sleep(6)  # Ждать n секунд перед следующим запросом
            return answer
        except Exception as e:
            print(f"Ошибка при обработке вопроса '{question}': {e}")
            retries += 1
            time.sleep(15)  # Дополнительная задержка перед повторной попыткой
    # Если после 3 попыток все равно возникает ошибка
    return "Я не знаю."

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))

In [ ]:
df.to_csv('../rag_answer/advanced_rag/html_split/final_4_gpt_3.5_var_2.csv', index=False)

+- у gpt получается ответить по конртексту как и ожидалось, но из-за маршрутизации случаются сбои и часть вопросов находится вне контекста

#### Giga generation stopped with reason: blacklist - 4 

## Variant №5
### Html_split + rag_fusion  + rerank 

In [9]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Advanced_RAG_5_gpt_3.5"

In [2]:
prompt_rag_fusion = ChatPromptTemplate.from_template("""
                        Вы - полезный помощник, который генерирует несколько поисковых запросов на основе одного входного запроса. \n
                        Постарайтесь разложить исходный вопрос - '{question}' на 2 более простых вопроса.\n 
                        Output (2 queries): \n""")

# Инициализация моделей
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

splits = split_documents_html(chunk_size=1800, chunk_overlap=100)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)

In [10]:
generate_queries_fusion = (
    prompt_rag_fusion |
    llm |
    StrOutputParser() |
    (lambda x: x.split("\n"))
)

# Определяем функцию для ранжирования документов
def retrieval_chain(question):
    documents = retriever.invoke(question)
    return rerank_doc(question, top_n=1, documents=documents)


# Итоговый промт
final_prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области пользовательских соглашений банка Tinkoff. Ваша задача — отвечать на вопросы клиентов на основе предоставленного контекста.\n
    Извлеките релевантную информацию из контекста, которая поможет ответить на вопрос.\n
    Сформулируйте ответ, который:
    - Является исчерпывающим и структурированным.
    - Не противоречит предоставленному контексту.
    - Краток и по делу.
    Если контекст не содержит достаточной информации, сообщите клиенту, что вы не знаете точного ответа, и предложите обратиться в банк для получения дополнительной информации.

    Question: {question} 
    Context: {context} 
    Answer:
""")


# объединение контекстов для размещения в promt
def remove_duplicate_contexts(question, context, step_back_context) -> str:
    all_contexts = context + step_back_context
    final_context = get_unique_documents(all_contexts)
    
    prompt = final_prompt.format(question=question, context=final_context)
    return prompt

# Функция для обработки двух вопросов и получения их результатов
def process_queries_with_retrieval(queries):
    results = []
    for query in queries:
        results.extend(retrieval_chain(query))
    return results

# Объединение всех шагов в одну цепочку
final_chain_5 = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retrieval_chain,
        "rag_fusion_context": generate_queries_fusion | RunnableLambda(lambda x: process_queries_with_retrieval(x)),
        "question": RunnablePassthrough()
    }
    | RunnableLambda(lambda x: remove_duplicate_contexts(x["question"], x["context"], x["rag_fusion_context"]))
    | llm
    | StrOutputParser()
)

# # Пример вызова
# response = final_chain_5.invoke({"question": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?"})
# print(response)

### Загрузка Датасета

In [11]:
import pandas as pd
import time
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки и повторных попыток
def process_question(question, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            answer = final_chain_5.invoke({"question": question})
            time.sleep(12)  # Ждать n секунд перед следующим запросом
            return answer
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            attempts += 1
            time.sleep(15)  # Подождать перед повторной попыткой
    print("Failed after multiple attempts.")
    return None

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))
display(df)

Error: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions". Retrying...
Error: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions". Retrying...
Error: peer closed connection without sending complete message body (incomplete chunked read). Retrying...
Error: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continu

,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,"Из предоставленного контекста следует, что пос..."
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...",Использование карты безопасно при соблюдении о...
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...",Хранение пин-кода карты в заметках на телефоне...
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...","Если вы потеряли карту, вам необходимо незамед..."
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,При использовании карты важно не расписываться...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,Если вы потеряли телефон с установленным мобил...
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...",Для совершения операций по карте в интернете р...
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,Для обезопасить себя от съема данных карты в б...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...",При бронировании номера в гостинице по карте т...


24 минуты работал алгоритм... - это несомнено минус большой


In [12]:
df.to_csv('../rag_answer/advanced_rag/html_split/Advanced_RAG_5_gpt_3.5.csv', index=False)

## Variant №6
### Html_split  + step_back  + rerank + promt (var 5)

In [13]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Advanced_RAG_6_gpt_3.5"

In [14]:
# Примеры для Few Shot
examples = [
    {"input": "Нужно ли ИП указывать свой сайт для предприятия?", "output": "Что необходимо предоставить ИП?"},
    {"input": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?", "output": "Что будет при неиспользовании кредитной карты?"},
    {"input": "Дают ли кредит на автомобиль?", "output": "На что дают кредит?"},
    {"input": "Распространяется ли страховое покрытие на смерть в алкогольном опьянении?", "output": "На что не распространяется страховое покрытие по риску 'Смерть'?"}
]

example_prompt = ChatPromptTemplate.from_messages([("human", "{input}"), ("ai", "{output}")])
few_shot_prompt = FewShotChatMessagePromptTemplate(example_prompt=example_prompt, examples=examples)

step_back_prompt = ChatPromptTemplate.from_messages([
    ("system", """Вы являетесь экспертом в области мировых знаний. Ваша задача - сделать шаг назад и перефразировать вопрос в более общий, на который проще ответить. Вот несколько примеров:"""),
    few_shot_prompt,
    ("user", "{question}")
])

# Инициализация моделей
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

splits = split_documents_html(chunk_size=1800, chunk_overlap=100)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [15]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)
# Определяем функцию для ранжирования документов
def retrieval_chain(question):
    documents = retriever.invoke(question)
    return rerank_doc(question, top_n=2, documents=documents)


# Генерация перефразированных вопросов
generate_queries_step_back = step_back_prompt | llm | StrOutputParser()

# Итоговый промт
final_prompt = ChatPromptTemplate.from_template("""
    Вы являетесь экспертом в области пользовательских соглашений банка Tinkoff. Ваша задача — отвечать на вопросы клиентов на основе предоставленного контекста.\n
    Извлеките релевантную информацию из контекста, которая поможет ответить на вопрос.\n
    Сформулируйте ответ, который:
    - Является исчерпывающим и структурированным.
    - Не противоречит предоставленному контексту.
    - Краток и по делу.
    Если контекст не содержит достаточной информации, сообщите клиенту, что вы не знаете точного ответа, и предложите обратиться в банк для получения дополнительной информации.

    Question: {question} 
    Context: {context} 
    Answer:
""")

def remove_duplicate_contexts(question, context, step_back_context) -> str:
    all_contexts = context + step_back_context
    final_context = get_unique_documents(all_contexts)
    
    prompt = final_prompt.format(question=question, context=final_context)
    return prompt

# Объединение всех шагов в одну цепочку
final_chain_6 = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retrieval_chain,
        "step_back_context": generate_queries_step_back | retrieval_chain,
        "question": RunnablePassthrough()
    }
    | RunnableLambda(lambda x: remove_duplicate_contexts(x["question"], x["context"], x["step_back_context"]))
    | llm
    | StrOutputParser()
)

# # Пример вызова
# response = final_chain_6.invoke({"question": "Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?"})
# print(response)

### Загрузка Датасета

In [16]:
import pandas as pd
import time
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки и повторных попыток
def process_question(question, max_attempts=3):
    attempts = 0
    while attempts < max_attempts:
        try:
            answer = final_chain_6.invoke({"question": question})
            time.sleep(6)  # Ждать n секунд перед следующим запросом
            return answer
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            attempts += 1
            time.sleep(15)  # Подождать перед повторной попыткой
    print("Failed after multiple attempts.")
    return None

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(lambda x: process_question(x.strip()))
display(df)

Error: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions". Retrying...
Error: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions". Retrying...
Error: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboar

,question,contexts,ground_truth,answer
0,Можно ли не расписываться на карте?,['После получения карты поставьте свою подпись...,При отсутствии подписи карта может быть не при...,"Из предоставленного контекста следует, что пос..."
1,Мой друг попросил меня дать свою карту для опл...,"['Никогда не принимайте рекомендации, советы, ...","Не передавайте карту другим лицам, в т.ч. родс...","Использование карты должно быть безопасным, ес..."
2,Безопасно ли хранить пинкод в заметках в телеф...,['Категорически запрещается записывать ПИН-код...,"Да, однако не указывайте в названии заметки, ч...",Хранение пин-кода в заметках на телефоне не яв...
3,Я потеряла карту,[' Вы потеряли карту или ее украли\nПрежде все...,"Если вы не уверены, что карту украли, не спеши...",При обнаружении утери или кражи вашей карты Ti...
4,Почему при использовании карты важно не распис...,"['Проверяйте правильность суммы, указанной на ...",Важно проверять правильность суммы перед подпи...,При использовании карты важно не расписываться...
5,"Что делать, если я потеряла телефон, на которо...","['При утрате Мобильного телефона, используемог...",Если вы потеряли телефон с установленным мобил...,"Если вы потеряли телефон, на котором было уста..."
6,Можно ли использовать интернет-кафе и другие о...,['Избегайте осуществления Интернет-операций с ...,"Нет, не рекомендуется использовать интернет-ка...",Для безопасности рекомендуется избегать соверш...
7,"Что делать, если я получил SMS-оповещение о оп...","['Если Вы уверены, что операция не могла быть ...","Если вы получили SMS-оповещение о операции, ко...",Если вы получили SMS-оповещение о операции по ...
8,Как обезопасить себя от съема данных карты в б...,"['Поэтому, при пользовании банкоматом не ленит...",Чтобы обезопасить себя от съема данных карты в...,Для обезопасить себя от съема данных карты в б...
9,"Правда ли, что при бронировании гостиницы по к...",['При бронировании номера в гостинице по карте...,"Нет, неправда. При бронировании гостиницы по к...",При бронировании номера в гостинице по карте н...


In [21]:
# Список номеров строк, которые нужно обработать
rows_to_process = [57]  # Укажите здесь нужные номера строк

# Применение функции только к указанным строкам
for index in rows_to_process:
    question = df.at[index, 'question'].strip()
    df.at[index, 'answer'] = process_question(question)

In [22]:
df.to_csv('../rag_answer/advanced_rag/html_split/Advanced_RAG_6_gpt_3.5.csv', index=False)